# **Triángulo de Jones-Romer**

En el 2010, Jones y Romer presentaron una segunda lista de regularidades empíricas sobre el crecimiento económico (una primera lista fue dada por Kaldor en 1961). Jones y Romer actualizaron el análisis con data de la post guerra (1960 en adelante).

Por regularidades empíricas se entiende a las premisas que resultan de las observaciones de hechos sobre el crecimiento económico. Una de las regularidades presentada por Jones y Romer es: 

> "Variation in modern growth rates. The variation in the rate of growth of per
capita GDP increases with the distance from the technology frontier" (Jones & Romer, 2010, p. 225).

Es decir, aquellos países más desarrollados (los que se encuentran en la 'frontera tecnológica') tienen tasas de crecimiento similares y pequeñas. Mientras que, los países menos desarrollados (lejos de la frontera) tienen una mayor divergencia en sus tasas. 


Gráficamente esta dispersión crea un triángulo entre las tasas de crecimiento y PBI de los países.
<img src="https://github.com/RominaGaribay/Trabajo-final/blob/main/image_1.png?raw=true" width="400">

# 0.Objetivo
**El objetivo de este trabajo es replicar graficamente esta premisa para cualquier año entre 1960 al 2020.** La información a utilizar será la data del Banco Mundial sobre:

  1. Tasa de crecimiento del PBI por países
  2. PBI per capital por país
  3. Clasificación de ingreso por país

Para poder replicar el triángulo se necesita ordenar esta data y ubicarla en los ejes del gráfico de dispersión.

In [1]:
import pandas as pd   
import wbgapi as wb

# 1.Tasa de crecimiento de PBI

> i) En el eje de las ordenadas se usa la información del promedio de la tasa de crecimiento per cápita de los países desde 1960 al 2020. Para ello se realizó un promedio simple de las tasas en cada país. La información fue recopilada en la columna Growth_prom.


In [2]:
#    1. Descargar data sobre tasas de crecimiento: ID:'NY.GDP.PCAP.KD.ZG' del df
g_PIB=wb.data.DataFrame('NY.GDP.PCAP.KD.ZG',labels=True) 
#g_PIB.head(5)

C:\Users\acwe\miniconda3\lib\site-packages\wbgapi\data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


### Promedio del crecimiento:

In [3]:
#    2. Obtener promedio por filas (cada fila es un país)
g_PIB['Growth_prom']=g_PIB.mean(axis=1,numeric_only=True)  
#g_PIB.head(5)

In [4]:
#    3. Limpiar data y mantener solo la tasa promedio 'Growth_prom'
g_PIB=g_PIB[['Country','Growth_prom']]
g_PIB=g_PIB.reset_index()
g_PIB.head(5)

,economy,Country,Growth_prom
0,ZWE,Zimbabwe,0.445556
1,ZMB,Zambia,0.173847
2,YEM,"Yemen, Rep.",-1.341735
3,PSE,West Bank and Gaza,1.516616
4,VIR,Virgin Islands (U.S.),-0.942663


# 2.PIB per cápita
> ii) Respecto al eje de las abscisas se coloca la información del PBI de todos los países como proporción al PBI de USA. Esto mide la distancia de los países de la 'frontera tecnológica' (que es USA). Por ejemplo, en el caso de Perú:
`Ratio pbi per cápita Perú=  (Pbi Perú Per Cápita)/(PBI EEUU Per Cápita)`
La misma operación se realizó con los demás países de la data. Los resultados se encuentra en la columna GDPcountry_USA.

> El año de elección de la información del PBI es cualquier año de 1960 en adelante (dado que Jones y Romer realizan su investigación con datos de la post guerra).



In [5]:
#    1. Descargar data sobre PBI per cápita: ID:'NY.GDP.PCAP.CD' del df 
PIB_percap=wb.data.DataFrame('NY.GDP.PCAP.CD')  
#PIB_percap.head(5)

C:\Users\acwe\miniconda3\lib\site-packages\wbgapi\data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


In [6]:
#    2. Cambiar nombre de columnas a solo el número de los años
PIB_percap.columns=list(range(1960,2021))    
#PIB_percap.head(5)

### En base a USA:
Elegir el año base (entre 1960 al 2020) de donde extraer el PIB de USA.

In [7]:
#    3. Elegir un año entre 1960 al 2000: AÑO BASE
import random
año=random.randint(1960,2020)     
año

2003

In [8]:
#    4. Localizar la observacion base 'PBI USA'
USA=PIB_percap.loc['USA']     
USA

1960     3007.123445
1961     3066.562869
1962     3243.843078
1963     3374.515171
1964     3573.941185
            ...     
2016    58021.400500
2017    60109.655726
2018    63064.418410
2019    65279.529026
2020    63593.443621
Name: USA, Length: 61, dtype: float64

In [9]:
#    5. Localizar la observación del PIB de USA en el año 'base'
USA=USA.loc[año]   
USA

39496.4858751381

In [10]:
def pais_USA(pais):
  '''
  Define el ratio entre el PBI per capita de un país y USA (en un año específico 'x'):
    input: PBI del país en un año específico x
    output: PBI del país como fracción al PBI de USA para un año específico x
  '''
  ratioo=pais/USA
  return ratioo

In [11]:
#    6. Aplicar la función 'pais_USA' al año elegido
PIB_percap['GDPcountry_USA']=PIB_percap[año].apply(lambda x:pais_USA(x)) 
#PIB_percap.head(5)

In [12]:
#    7. Crear columna con el PBI de USA en el año elegido
PIB_percap['USA_base']=USA  
#PBI_percap

In [13]:
#    8. Limpiar data y mantener el ratio de PBI/USA: 'GDPcountry_USA'
PIB_percap=PIB_percap[[año, 'USA_base', 'GDPcountry_USA' ]] 
#PIB_percap.head(5)

In [14]:
#    9. Ordenar índices y renombrar columnas
PIB_percap.columns=['GDPcountry_'+str(año)]+list(PIB_percap.columns[1:])
PIB_percap=PIB_percap.reset_index()
PIB_percap.head(5)

,economy,GDPcountry_2003,USA_base,GDPcountry_USA
0,ABW,20833.976357,39496.485875,0.527489
1,AFE,819.743981,39496.485875,0.020755
2,AFG,190.683814,39496.485875,0.004828
3,AFW,706.955063,39496.485875,0.017899
4,AGO,982.961008,39496.485875,0.024887


# 3.BASE de datos final:
Se recopila la información de países: promedio de crecimiento (1960-2020), ratio de PBI per capita para un año entre (1960-2020) y clasificación del país segun nivel de ingreso.
Se usa la función 'merge'.

In [15]:
#    1. Unir base de tasa de crecimiento y PBI per cápita
PIB=pd.merge(g_PIB,PIB_percap,on='economy',how='inner')   
#PIB.head(5)

In [16]:
#    2. Descargar data con descripción del país según nivel de ingreso: 'income_classification' y region: 'Region'
lst1=wb.economy.info().table()  
lst2=wb.income.info().table() 

In [17]:
df1=pd.DataFrame(data=lst1,columns=["economy", "Country", "Region", 'income_classification'])
df2=pd.DataFrame(data=lst2,columns=["income_classification", "income_classif" ])

In [18]:
df=pd.merge(df1,df2,on='income_classification',how='inner')

In [19]:
#    3. Unir base del PIB y decripción del país
PIB=pd.merge(PIB,df,on=['economy', 'Country'],how='inner')

In [20]:
PIB=PIB.drop(PIB[PIB['Region']==''].index)

### Base de datos final

In [21]:
PIB #BASE DE DATOS FINAL

,economy,Country,Growth_prom,GDPcountry_2003,USA_base,GDPcountry_USA,Region,income_classification,income_classif
0,ZWE,Zimbabwe,0.445556,478.007604,39496.485875,0.012103,SSF,LMC,Lower middle income
1,ZMB,Zambia,0.173847,435.458199,39496.485875,0.011025,SSF,LMC,Lower middle income
2,YEM,"Yemen, Rep.",-1.341735,620.382726,39496.485875,0.015707,MEA,LIC,Low income
3,PSE,West Bank and Gaza,1.516616,1257.698570,39496.485875,0.031843,MEA,LMC,Lower middle income
4,VIR,Virgin Islands (U.S.),-0.942663,31731.256624,39496.485875,0.803394,LCN,HIC,High income
...,...,...,...,...,...,...,...,...,...
212,AND,Andorra,-0.000741,32272.845891,39496.485875,0.817107,ECS,HIC,High income
213,ASM,American Samoa,-0.309695,8807.462812,39496.485875,0.222994,EAS,UMC,Upper middle income
214,DZA,Algeria,1.195406,2103.381291,39496.485875,0.053255,MEA,LMC,Lower middle income
215,ALB,Albania,2.633886,1846.120121,39496.485875,0.046741,ECS,UMC,Upper middle income


# 4.Gráfico final
Usando altair

In [22]:
import altair as alt

In [23]:
chart=alt.Chart(PIB).mark_circle().encode(alt.X('GDPcountry_USA:Q',
                                                title='PIB per cápita como fracción de USA para el año '+ f'{año}'),
                                          alt.Y('Growth_prom:Q',
                                                title='Crecimiento (%) PBI 1960-2020'),
                                          color=alt.condition(alt.datum.economy == 'USA',
                                                              alt.value('black'),
                                                              alt.Color('income_classif:N',
                                                                        title='Grupos de ingreso',
                                                                        scale=alt.Scale(scheme='rainbow'))),
                                          tooltip=['Country',
                                                   'GDPcountry_USA',
                                                   'Growth_prom']).interactive().properties(title={'text':['TRIÁNGULO DE JONES ROMER'],
                                                                                                   'subtitle':['año base '+ f'{año}']},
                                                                                            width=800,height=500)                                          
                                          

### Gráfico final

In [24]:
chart=chart.configure_legend(strokeColor='grey',fillColor='#d1dce6',padding=20,cornerRadius=20,orient='top-right')
chart=chart.configure_axis(titleColor='grey',titleFontSize=20)
chart.configure_title(fontSize=33,font='Courier',anchor='middle',color='grey',orient='top',subtitleColor='grey',
                      subtitleFont='Courier',subtitleFontSize=18,)

alt.Chart(...)

# Conclusiones
El gráfico muestra la regularidad empírica enunciada por Jones y Romer para un año aleatorio entre 1960 al 2020.

**Los países con ingresos más altos (puntos rosados) tienen tasas de crecimiento bajas y cercanas al 2%. Esta convergencia es mayor cuando se está más cerca a la frontera tecnológica (tomando como referencia USA, punto negro). Los países cuyos ingresos son bajos (puntos naranjas) o mediano-bajos (puntos verdes) tienen tasas de crecimiento más dipersas (entre -2% al 8%).**

Las razones se explican con ayuda de teorías del crecimiento:

* Los países con niveles de PBI per cápita menores tienen una mayor senda de crecimiento por recorrer. Por tanto, los 'saltos' de las tasas PBI per cápita son grandes.
* Los países de mayores ingresos estarían cerca a su estado estacionario. 
* Las tasas de crecimiento son menores en los países de grandes ingresos (en el gráfico del 1 o 2%) debido a los retornos decrecientes de su capital: al tener altos niveles de capital, el capital adicional retorna menos producto por capital invertido. Por ello, se vería una convergencia entre sus tasas.
* Que las tasas de crecimiento converjan no implica que los países con grandes ingresos empobrezcan: aunque el *crecimiento* de su PBI per cápita será menor, el *nivel* ya es alto.

*Estado estacionario*: Es cuando en una economía se usa de la forma más eficiente sus recursos. En este punto, no se puede obtener un nivel mayor de producción si que la depreciación supere al ahorro/inversión.

### Bibliografía:
Jones, Charles I., and Paul M. Romer. 2010. "The New Kaldor Facts: Ideas, Institutions, Population, and Human Capital." American Economic Journal: Macroeconomics, 2 (1): 224-45. DOI: 10.1257/mac.2.1.224